# Stick Game

## The class to play

In [26]:
nbMaxTook = 3
class stateSG():

    def __init__(self, nbStick, maxPlayer):
        self.nbStick = nbStick
        self.maxPlayer = 1 if maxPlayer==True else -1
        
    def calculateScore(self):
        
        if(self.nbStick <= 1):
            return -self.maxPlayer 
        return 0 #To indicate no one win for now. Can be replace by a function which estimate the score
    
    def getChoices(self):
        nbMaxTurn = nbMaxTook if nbMaxTook<= self.nbStick else self.nbStick
        return [i for i in range(1, nbMaxTurn+1)]
        
    def doChoice(self, nb, inNewState = False):
        
        if(inNewState == False):
            self.nbStick -= nb
            self.maxPlayer *= -1
            return None
        else:
            return stateSG(self.nbStick-nb, -self.maxPlayer)
            
    def undoChoice(self, nb, inNewState = False):

        return self.doChoice(-nb, inNewState)
    
    def toKey(self):
        
        return str(self.nbStick)+";"+str(self.maxPlayer)

## Minmax implementation (without memoization)

In [5]:
def minmaxDecision(state):
    
    score = state.calculateScore()
    if(score != 0):
        return score
    
    choices = state.getChoices()
    if(state.maxPlayer == 1):
        score = -1
        for choice in choices:
            state.doChoice(choice)
            score = max(score, minmaxDecision(state))
            state.undoChoice(choice)
        return score
    else:
        score = 1
        for choice in choices:
            state.doChoice(choice)
            score = min(score, minmaxDecision(state))
            state.undoChoice(choice)
        return score

In [13]:
def minmaxDecision(state):
    
    score = state.calculateScore()
    if(score != 0):
        return score, None
    
    choices = state.getChoices()
    if(state.maxPlayer == 1):
        score = -1
        bestChoice = None
        for choice in choices:
            state.doChoice(choice)
            newScore, newChoice = minmaxDecision(state)
            if(newScore > score):
                score = newScore
                bestChoice = choice
            state.undoChoice(choice)
        return score, bestChoice
    else:
        score = 1
        bestChoice = None
        for choice in choices:
            state.doChoice(choice)
            newScore, newChoice = minmaxDecision(state)
            if(newScore < score):
                score = newScore
                bestChoice = choice
            state.undoChoice(choice)
        return score, bestChoice

In [15]:
for i in range(1, 11):
    test = stateSG(i, True)
    score, choice = minmaxDecision(test) 
    print("Start with ", test.nbStick, " and win with best choice :"+str(choice) if score==1 else " and lose")

Start with  1  and lose
Start with  2  and win with best choice :1
Start with  3  and win with best choice :2
Start with  4  and win with best choice :3
Start with  5  and lose
Start with  6  and win with best choice :1
Start with  7  and win with best choice :2
Start with  8  and win with best choice :3
Start with  9  and lose
Start with  10  and win with best choice :1


## Negamax implementation (without memoization)

In [9]:
def negamaxDecision(state):
    
    score = state.calculateScore()
    if(score != 0):
        return score * state.maxPlayer
    
    score = -1
    choices = state.getChoices()
    for choice in choices:
        state.doChoice(choice)
        score = max(score,  -negamaxDecision(state))
        state.undoChoice(choice)
    return score

In [20]:
def negamaxDecision(state):
    
    score = state.calculateScore()
    if(score != 0):
        return score*state.maxPlayer, None
    
    score = -1
    bestChoice = None
    choices = state.getChoices()
    for choice in choices:
        state.doChoice(choice)
        newScore, newChoice = negamaxDecision(state)
        newScore *= -1
        if(newScore > score):
            score = newScore
            bestChoice = choice
        state.undoChoice(choice)
    return score, bestChoice

In [21]:
for i in range(1, 11):
    test = stateSG(i, True)
    score, choice = negamaxDecision(test)
    print("Start with ", test.nbStick, " and win with best choice :"+str(choice) if score==1 else " and lose")

Start with  1  and lose
Start with  2  and win with best choice :1
Start with  3  and win with best choice :2
Start with  4  and win with best choice :3
Start with  5  and lose
Start with  6  and win with best choice :1
Start with  7  and win with best choice :2
Start with  8  and win with best choice :3
Start with  9  and lose
Start with  10  and win with best choice :1


## Algorithms with Memoization

In [32]:
states = {}

## Minmax implementation (with memoization)

In [29]:
def minmaxDecision(state):
    
    global states
    key = state.toKey()
    if(key in states):
        return states[key]
    
    score = state.calculateScore()
    if(score != 0):
        states[key] = score, None
        return states[key]
    
    choices = state.getChoices()
    if(state.maxPlayer == 1):
        score = -1
        bestChoice = None
        for choice in choices:
            state.doChoice(choice)
            newScore, newChoice = minmaxDecision(state)
            if(newScore > score):
                score = newScore
                bestChoice = choice
            state.undoChoice(choice)
        states[key] = score, bestChoice
    else:
        score = 1
        bestChoice = None
        for choice in choices:
            state.doChoice(choice)
            newScore, newChoice = minmaxDecision(state)
            if(newScore < score):
                score = newScore
                bestChoice = choice
            state.undoChoice(choice)
        states[key] = score, bestChoice
    return states[key]

In [30]:
for i in range(1, 11):
    test = stateSG(i, True)
    score, choice = minmaxDecision(test) 
    print("Start with ", test.nbStick, " and win with best choice :"+str(choice) if score==1 else " and lose")

Start with  1  and lose
Start with  2  and win with best choice :1
Start with  3  and win with best choice :2
Start with  4  and win with best choice :3
Start with  5  and lose
Start with  6  and win with best choice :1
Start with  7  and win with best choice :2
Start with  8  and win with best choice :3
Start with  9  and lose
Start with  10  and win with best choice :1


## Negamax implementation (with memoization)

In [33]:
def negamaxDecision(state):
    
    global states
    key = state.toKey()
    if(key in states):
        return states[key]
    
    score = state.calculateScore()
    if(score != 0):
        states[key] = score*state.maxPlayer, None
        return states[key] 
    
    score = -1
    bestChoice = None
    choices = state.getChoices()
    for choice in choices:
        state.doChoice(choice)
        newScore, newChoice = negamaxDecision(state)
        newScore *= -1
        if(newScore > score):
            score = newScore
            bestChoice = choice
        state.undoChoice(choice)
    states[key] = score, bestChoice
    return states[key]

In [34]:
for i in range(1, 11):
    test = stateSG(i, True)
    score, choice = negamaxDecision(test)
    print("Start with ", test.nbStick, " and win with best choice :"+str(choice) if score==1 else " and lose")

Start with  1  and lose
Start with  2  and win with best choice :1
Start with  3  and win with best choice :2
Start with  4  and win with best choice :3
Start with  5  and lose
Start with  6  and win with best choice :1
Start with  7  and win with best choice :2
Start with  8  and win with best choice :3
Start with  9  and lose
Start with  10  and win with best choice :1
